<h2>Project 1: $k$-Nearest Neighbors</h2>
<p><cite><center>So many points,<br>
some near some far,<br>
- who are my true neighbors?</center></cite></p>

<h3>Introduction</h3>

<p>In this project, you will build a $k$-nearest neighbor classifier.</p>

<strong>How to submit:</strong> You can submit your code using the red <strong>Submit</strong> button above. This button will send any code below surrounded by <strong>#&lt;GRADED&gt;</strong><strong>#&lt;/GRADED&gt;</strong> tags below to the autograder, which will then run several tests over your code. By clicking on the <strong>Details</strong> dropdown next to the Submit button, you will be able to view your submission report once the autograder has completed running. This submission report contains a summary of the tests you have failed or passed, as well as a log of any errors generated by your code when we ran it.

Note that this may take a while depending on how long your code takes to run! Once your code is submitted you may navigate away from the page as you desire -- the most recent submission report will always be available from the Details menu.

<p><strong>Evaluation:</strong> Your code will be autograded for technical
correctness and--on some assignments--speed. Please <em>do not</em> change the names of any provided functions or classes within the code, or you will wreak havoc on the autograder. Furthermore, <em>any code not surrounded by <strong>#&lt;GRADED&gt;</strong><strong>#&lt;/GRADED&gt;</strong> tags will not be run by the autograder</em>. However, the correctness of your implementation -- not the autograder's output -- will be the final judge of your score.  If necessary, we will review and grade assignments individually to ensure that you receive due credit for your work.

<p><strong>Academic Integrity:</strong> We will be checking your code against other submissions in the class for logical redundancy. If you copy someone else's code and submit it with minor changes, we will know. These cheat detectors are quite hard to fool, so please don't try. We trust you all to submit your own work only; <em>please</em> don't let us down. If you do, we will pursue the strongest consequences available to us.

<p><strong>Getting Help:</strong> You are not alone!  If you find yourself stuck  on something, contact the course staff for help.  Office hours, section, and the <a href="https://piazza.com/class/icxgflcnpra3ko">Piazza</a> are there for your support; please use them.  If you can't make our office hours, let us know and we will schedule more.  We want these projects to be rewarding and instructional, not frustrating and demoralizing.  But, we don't know when or how to help unless you ask.  



**Libraries**: Before we get started we need to install a few libraries. You can do this by executing the following code.

In [ ]:
#<GRADED>
import numpy as np
from scipy.stats import mode
#</GRADED>
import sys
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
from scipy.io import loadmat
import time
from helper_functions import loaddata, visualize_knn_2D, visualize_knn_images, plotfaces, visualize_knn_boundary


print('You\'re running python %s' % sys.version.split(' ')[0])

<h3> k-Nearest Neighbors implementation in Python </h3>

<p>Our first goal towards a $k$NN classifier is to build a classifier for handwritten digits classification and face recognition. 
</p>

**Data:** We first obtain some data for testing your code. The data resides in the files <code>faces.mat</code> and <code>digits.mat</code> which hold the datasets for the further experiments.

Here, <b>xTr</b> are the training vectors with labels <b>yTr</b> and <b>xTe</b> are the testing vectors with labels <b>yTe</b>. 
As a reminder, to predict the label or class of an image in <b>xTe</b>, we will look for the <i>k</i>-nearest neighbors in <b>xTr</b> and predict a label based on their labels in <b>yTr</b>. For evaluation, we will compare these labels against the true labels provided in <b>yTe</b>.</p>

<h4> Visualizing data</h4>

Let us take a look at our data. The following script will take the first 10 training images from the face data set and visualize them.

In [ ]:
xTr,yTr,xTe,yTe=loaddata("faces.mat")

plt.figure()
plotfaces(xTr[:9, :])


<h4> Implementation </h4>
<p> The following questions will ask you to finish these functions in a pre-defined order. <br></p>

<p>(a) Implement the function  <b><code>l2distance</code></b>. You may use your own code(s) from the previous project.</p>


In [ ]:
#<GRADED>
def l2distance(X,Z=None):
    # function D=l2distance(X,Z)
    #
    # Computes the Euclidean distance matrix.
    # Syntax:
    # D=l2distance(X,Z)
    # Input:
    # X: nxd data matrix with n vectors (rows) of dimensionality d
    # Z: mxd data matrix with m vectors (rows) of dimensionality d
    #
    # Output:
    # Matrix D of size nxm
    # D(i,j) is the Euclidean distance of X(i,:) and Z(j,:)
    #
    # call with only one input:
    # l2distance(X)=l2distance(X,X)
    #

    if Z is None:
        Z=X;

    n,d1=X.shape
    m,d2=Z.shape
    assert (d1==d2), "Dimensions of input vectors must match!"

   
    S = np.diag(X @ (X.T)).reshape(n, 1)
    R = np.diag(Z @ (Z.T)).reshape(1,m)
    D = np.sqrt(-2 * (X @ Z.T) + S + R)

    return D
#</GRADED>


<p>(b) Implement the function <b><code>findknn</code></b>, which should find the $k$ nearest neighbors of a set of vectors within a given training data set. The call of 
<pre>
 [I,D]=findknn(xTr,xTe,k);
</pre> 
should result in two matrices $I$ and $D$, both of dimensions $k\times n$, where $n$ is the number of input vectors in <code>xTe</code>. The matrix $I(i,j)$ is the index of the $i^{th}$ nearest neighbor of the vector $xTe(j,:)$. 
So, for example, if we set <code>i=I(1,3)</code>, then <code>xTr(i,:)</code> is the first nearest neighbor of vector <code>xTe(3,:)</code>. The second matrix $D$ returns the corresponding distances. So $D(i,j)$ is the distance of $xTe(j,:)$ to its $i^{th}$ nearest neighbor.
</p>

In [ ]:
#<GRADED>
def findknn(xTr,xTe,k):
    """
    function [indices,dists]=findknn(xTr,xTe,k);
    
    Finds the k nearest neighbors of xTe in xTr.
    
    Input:
    xTr = nxd input matrix with n row-vectors of dimensionality d
    xTe = mxd input matrix with m row-vectors of dimensionality d
    k = number of nearest neighbors to be found
    
    Output:
    indices = kxm matrix, where indices(i,j) is the i^th nearest neighbor of xTe(j,:)
    dists = Euclidean distances to the respective nearest neighbors
    """
    # Enter your code here
    D = l2distance(xTr, xTe)
    indices = (D.argsort(axis=0))[:k, :]
    D.sort(axis=0)
    dists = D[:k, :]
    #raise NotImplementedError('Your code goes here!')

    return indices, dists
    # until here

#</GRADED>

<p> The following demo samples random points in 2D. If your findknn  function is correctly implemented, you should be able to click anywhere on the plot to add a test point. The function should then draw direct connections from your test point to the k  nearest neighbors. Verify manually if your code is correct.
</p>

In [ ]:
visualize_knn_2D(findknn)

We can visualize the k=3 nearest training neighbors of some of the test points (Click on the image to cycle through different test points). 

In [ ]:
visualize_knn_images(findknn, imageType='faces')

<p>(c) The function <b><code>analyze</code></b> should compute various metrics to evaluate a classifier. The call of
<pre>
  result=analyze(kind,truth,preds);
</pre>
should output the <b>accuracy</b> or <b>absolute loss</b> in variable <code>result</code>. The type of output required can be specified in the input argument <code>kind</code> as <code>"abs"</code> or <code>"acc"</code>. The input variables <code>truth</code> and <code>pred</code> should contain vectors of true and predicted labels respectively.
For example, the call
<pre>
>> analyze('acc',[1 2 1 2],[1 2 1 1])
</pre>
should return an accuracy of 0.75. Here, the true labels are 1,2,1,2 and the predicted labels are 1,2,1,1. So the first three examples are classified correctly, and the last one is wrong --- 75% accuracy.
</p>



In [ ]:
#<GRADED>
def analyze(kind,truth,preds):
    """
    function output=analyze(kind,truth,preds)         
    Analyses the accuracy of a prediction
    Input:
    kind='acc' classification error
    kind='abs' absolute loss
    (other values of 'kind' will follow later)
    """
    
    truth = truth.flatten()
    preds = preds.flatten()
    
    if kind == 'abs':
        output=(truth==preds).sum()
    elif kind == 'acc':
        output=(truth==preds).sum()/truth.size
    
    return output

#</GRADED>


<p>(e) Implement the function <b><code>knnclassifier</code></b>, which should perform $k$ nearest neighbor classification on a given test data set. The call <pre>preds=knnclassifier(xTr,yTr,xTe,k)</pre>
should output the predictions for the data in <code>xTe</code> i.e. <code>preds[i]</code> will contain the prediction for <code>xTe[i,:]</code>.</p>

In [ ]:
#<GRADED>
        
def knnclassifier(xTr,yTr,xTe,k):
    """
    function preds=knnclassifier(xTr,yTr,xTe,k);
    
    k-nn classifier 
    
    Input:
    xTr = nxd input matrix with n row-vectors of dimensionality d
    xTe = mxd input matrix with m row-vectors of dimensionality d
    k = number of nearest neighbors to be found
    
    Output:
    
    preds = predicted labels, ie preds(i) is the predicted label of xTe(i,:)
    """

    # your code goes here
    
    # fix array shapes
    n, d = xTr.shape
    yTr = (yTr.flatten()).reshape(n, 1)

    #shuffle the training examples
    training = np.hstack((xTr, yTr))
    np.random.shuffle(training)
    yTr = training[:, -1]

    #flatten again
    yTr = yTr.flatten()
    m, d = xTe.shape

    #allocate prediction array
    preds = np.zeros(m)
    
    #get k nearest neighbors indices and their distances to each test example
    indices, dists = findknn((training[:, :-1]).reshape(n, d), xTe, k)
    label_matrix = np.take(yTr, indices)

    for i in range(m):
        #if k=1, then there is no need to worry about ties among nearest neighbors' labels
        if k == 1:
            preds[i] = label_matrix[0, i]
        #if not, then we have to be careful to deal with the ties among labels of neighbors
        else:
            unique, count = np.unique(label_matrix[:, i], return_counts=True)
            unique = unique[count.argsort()]
            count.sort()
            #if there exists only one label in k labels or every label is different, we pick that label
            if count.size == 1:
                preds[i] = unique[-1]
            elif count[-1]==1:
                preds[i]=label_matrix[0,i]
            #check if there exists a tie between two most frequent labels among k labels
            elif count[-1] == count[-2]:
                reduced = 1
                #progressively discard the most distant neighbor's label until tie breaks
                while reduced < k - 1:
                    unique, count = np.unique(label_matrix[:-reduced, i], return_counts=True)
                    unique = unique[count.argsort()]
                    count.sort()
                    if count.size == 1:
                        break
                    elif count[-1] == count[-2]:
                        reduced = reduced + 1
                    else:
                        break
                preds[0] = unique[-1]
            else:
                preds[0] = unique[-1]
    #raise NotImplementedError('Your code goes here!')
    #add this later preds=preds.astype(int)
    preds=preds.flatten()
    return preds

#</GRADED>

<p>You can compute the actual classification error on the test set by calling
<pre>
>> analyze("acc",yTe,knnclassifier(xTr,yTr,xTe,3))
</pre></p>

<p>(e) This script runs the $k$-nearest neighbor classifier over the faces and digits data set. The faces data set has $40$ classes, the digits data set $10$. What classification accuracy would you expect from a random classifier?</p>

In [ ]:
print("Face Recognition: (1-nn)")
xTr,yTr,xTe,yTe=loaddata("faces.mat") # load the data
t0 = time.time()
preds = knnclassifier(xTr,yTr,xTe,1)
result=analyze("acc",yTe,preds)
t1 = time.time()
print("You obtained %.2f%% classification acccuracy in %.4f seconds\n" % (result*100.0,t1-t0))

#### KNN Boundary Visualization
To help give you a visual understanding of how the KNN boundary is affected by $k$ and the specific dataset, feel free to play around with the visualization below. Click anywhere in the graph to add a negative class point. Hold down 'p' and click anywhere to add a positive class point. To increase $k$ hold down 'h' and click anywhere.

In [ ]:
visualize_knn_boundary(knnclassifier)

<p>(f) (optional) Sometimes a $k$-NN classifier can result in a draw, when the majority vote is not clearly defined. Can you improve your accuracy by falling back onto $k$-NN with lower $k$ in such a case?</p>

<p>(g) Edit the function <b><code>competition</code></b>, which reads in a training and testing set and makes predictions. Inside this function you are free to use any combination or variation of the k-nearest neighbor classifier. Can you beat my submission on our secret training and testing set? </p>

<h4>Evaluation</h4>
<p>For this project, you will be ranked on the following measures:
<ul>
<li>Percentage of test cases passed</li>
<li>Average of:
<ul>
  <li>Accuracy on the faces test dataset and</li>
  <li>Accuracy on the digits test dataset</li>
  <li>Accuracy on the <i>secret</i> test dataset</li>
</ul>
</ul>

</p>

Let's take a look at the competition data. We will use plotdata function to visualize the first 10 training images from the digit data set and visualize them. Click on images to cycle through the competition data.

In [ ]:
visualize_knn_images(findknn, imageType='digits')

In [ ]:
#<GRADED>
def competition(xTr,yTr,xTe):
    """
    function preds=competition(xTr,yTr,xTe);
    
    A classifier that outputs predictions for the data set xTe based on 
    what it has learned from xTr,yTr
    
    Input:
    xTr = nxd input matrix with n column-vectors of dimensionality d
    xTe = mxd input matrix with n column-vectors of dimensionality d
    
    Output:
    
    preds = predicted labels, ie preds(i) is the predicted label of xTe(i,:)
    """
    n, d = xTr.shape
    if n == 1:
        preds = knnclassifier(xTr, yTr, xTe, 1)
    elif n > 1:
        yTr = yTr.flatten()
        yTr = yTr.reshape(n, 1)

        # shuffle training examples
        shuffled = np.hstack((xTr, yTr))
        np.random.shuffle(shuffled)
        xTr = (shuffled[:, :-1]).reshape(n, d)
        yTr = shuffled[:, -1].reshape(n, 1)

        #randomly splitting given training data as 80% training examples and 20% CV examples to pick optimal k
        #it is extremely unlikely that training data set will only have one training example, but just in case
        new_n = int(max(np.floor(n * 0.8), 1))
        train_new = xTr[:new_n, :]
        labels_new = yTr[:new_n, :]
        test_data_new = xTr[new_n:, :]
        test_label_new = yTr[new_n:, :]
        m, d = xTe.shape
        preds_matrix = np.zeros((15, n - new_n + 1))
        #for every k from 1-nn to 15-nn find the optimal k-nn that minimizes the relative error
        for k in range(15):
            preds_matrix[k, 1:] = knnclassifier(train_new, labels_new, test_data_new, k + 1)
            preds_matrix[k, 0] = analyze("acc", test_label_new, preds_matrix[k, 1:])
        row = np.argmax(preds_matrix[:, 0])
        #finally pick the optimal k and use them with test examples
        preds = knnclassifier(xTr, yTr, xTe, row + 1)
    return preds
#</GRADED>